In [1]:
!pip install PyGithub

  Using cached PyGithub-2.1.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached PyNaCl-1.5.0-cp36-abi3-macosx_10_10_universal2.whl (349 kB)
Using cached PyGithub-2.1.1-py3-none-any.whl (302 kB)


In [28]:
import os
from github import Github

github_token = os.environ.get('GITHUB_ACCESS_TOKEN')

# using an access token
g = Github(github_token)

In [29]:
def get_repo(repo_name):
    repo = f"{repo_name}"
    print(repo)
    repo = g.get_user().get_repo(repo)
    print(repo)
    return repo

Create file, commit and push

In [30]:
from github import Github
from github.GithubException import GithubException

def create_or_update_file_in_repo(repo, file_path, file_contents, commit_message):
    """
    Create or update a file in a GitHub repository.

    Args:
    github_token (str): Your GitHub access token.
    repo_name (str): Name of the repository.
    file_path (str): Path of the file in the repository.
    file_contents (str): Contents to be written to the file.
    commit_message (str): Commit message.

    Returns:
    None
    """

    try:
        # Try to get the file
        contents = repo.get_contents(file_path)
        # File exists, update it
        repo.update_file(contents.path, commit_message, file_contents, contents.sha)
        print("File updated successfully.")
    except GithubException as e:
        if e.status == 404:
            # File does not exist, create a new one
            repo.create_file(file_path, commit_message, file_contents)
            print("File created successfully.")
        else:
            # Some other error occurred
            print(f"An error occurred: {e}")
    
    return "File commited"

# Create the file in the repository
file_path = "test_create.txt"
file_contents = "testing github api"
commit_message = "testing"

repo = get_repo(repo_name="oreilly_live_training_agents")
create_or_update_file_in_repo(repo, file_path, file_contents, commit_message)

oreilly_live_training_agents
Repository(full_name="EnkrateiaLucca/oreilly_live_training_agents")
File created successfully.


Delete file, commit and push

In [31]:


def delete_file_in_repo(repo, file_path, commit_message):
    """
    Delete a file in a GitHub repository.

    Args:
    github_token (str): Your GitHub access token.
    repo_name (str): Name of the repository.
    file_path (str): Path of the file in the repository.
    commit_message (str): Commit message.

    Returns:
    None
    """
    try:
        # Try to get the file
        contents = repo.get_contents(file_path)
        # Delete the file
        repo.delete_file(contents.path, commit_message, contents.sha)
    except GithubException as e:
        if e.status == 404:
            # File does not exist
            print("File does not exist")
        else:
            # Some other error occurred
            print(f"An error occurred: {e}")
    
    return "File deleted"


file_path = "test_create.txt"
commit_message = "test deletion file"
delete_file_in_repo(repo, file_path, commit_message)

'File deleted'

In [37]:
import git

def clone_repo(repo_host_and_name, access_token, clone_dir):
    """
    Clones a repository given a GitHub URL using PyGithub and GitPython.

    :param github_url: URL of the GitHub repository
    :param access_token: GitHub Access Token for authentication
    :param clone_dir: Directory where the repository will be cloned
    """
    # Initialize GitHub instance with access token
    g = Github(access_token)
    
    # Get the repository
    repo = g.get_repo(repo_host_and_name)

    # Clone the repository
    git.Repo.clone_from(repo.clone_url, os.path.join(clone_dir, repo_host_and_name), env={"GIT_SSL_NO_VERIFY": "1"})

# Example usage
clone_repo("EnkrateiaLucca/AI_Learning_ToolKit",
           github_token, "./")

In [38]:
!ls -d */

EnkrateiaLucca/   dev-notebooks/    test/
assets-resources/ langchain_agents/


In [39]:
def create_repo(repo_name, access_token):
    """
    Creates a repository given a GitHub URL using PyGithub and GitPython.

    :param github_url: URL of the GitHub repository
    :param access_token: GitHub Access Token for authentication
    :param clone_dir: Directory where the repository will be cloned
    """
    # Initialize GitHub instance with access token
    g = Github(access_token)
    
    # Get the repository
    repo = g.get_user().create_repo(repo_name)
    
    return repo

repo = create_repo("test_repo", github_token)

Repository(full_name="EnkrateiaLucca/test_repo")

In [46]:
repo = g.get_user().get_repo("test_repo")
create_or_update_file_in_repo(repo, file_path, file_contents, commit_message)

File created successfully.


In [47]:
delete_file_in_repo(repo, file_path, commit_message)

'File deleted'

In [48]:
def delete_repo(repo_name, access_token):
    """
    Deletes a repository given a GitHub URL using PyGithub and GitPython.

    :param github_url: URL of the GitHub repository
    :param access_token: GitHub Access Token for authentication
    :param clone_dir: Directory where the repository will be cloned
    """
    # Initialize GitHub instance with access token
    g = Github(access_token)
    
    # Get the repository
    repo = g.get_user().get_repo(repo_name)
    
    repo.delete()

    return "Repo deleted"

delete_repo("test_repo", github_token)

'Repo deleted'

In [50]:
from openai import OpenAI
import re
client = OpenAI()


def extract_python_code(output):
    # Use regular expressions to find Python code within the output string
    pattern = r"```python\n(.*?)```"
    matches = re.findall(pattern, output, re.DOTALL)
    
    # Return the extracted Python code as a list
    return matches


def write_code(task_description):
    """
    Write code for a task.

    :param task_description: Description of the task
    """
    
    SYSTEM_PROMPT = f"You are an expert AI and machine learning engineer. You are tasked with \
        writing code for various tasks. The code you write should be well documented, clean,modular\
        and precise."
    
    PROMPT_CODE = f"Write code for the following task:\n\n{task_description}\n\n The output should \
        only be Python code. \n\n Output code:"
    
    response = client.chat.completions.create(
    temperature=0, 
    model="gpt-4-1106-preview",
    messages=[{"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content":PROMPT_CODE}]
    )
    
    return response.choices[0].message.content     

with open("sample_app_to_test_github_agent.py", "r") as f:
    script_contents = f.read()

output_code = write_code(f"Given this Python script: {script_contents}, I want you to write a script that:\n1. Tests the script and saves the outupts to a log_test.txt file. \n2. Review your script to make sure it will run")
formatted_code = extract_python_code(output_code)[0]
print(formatted_code)

To test the given script and save the outputs to a `log_test.txt` file, we need to create a test script that will run the `start_emotion_app` function and capture its output. However, since the original script is designed to work with a webcam and display live video, we need to modify it to work in a test environment. We will simulate the webcam input with a static image and redirect the standard output and error to a log file.

Here's a Python script that does this:

```python
import sys
import torch
import cv2 as cv
from io import StringIO
from contextlib import redirect_stdout, redirect_stderr

# Import the original script's components (assuming the script is named original_script.py)
from original_script import Net, prep_image, start_emotion_app, label_map

# Function to simulate the webcam capture with a static image
def simulate_webcam_capture(image_path):
    image = cv.imread(image_path)
    if image is None:
        raise ValueError("Image not found or unable to read.")
    re

In [ ]:
import argparse
from langchain.chat_models import ChatOpenAI
from langchain.tools import tool
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool

def main():
    parser = argparse.ArgumentParser(description='GitHub Automation')
    parser.add_argument('command', choices=['commit', 'create_file', 'read_file'], help='The GitHub command')
    parser.add_argument('-m', '--message', help='The commit message')
    parser.add_argument('-c', '--contents', help='The contents of the file')
    parser.add_argument('-p', '--path', help='The file path')
    
    args = parser.parse_args()
    
    llm = ChatOpenAI(temperature=0)
    tools = [github_commit_tool, create_file_tool, read_file_tool]
    agent_executor = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
    
    if args.command == 'commit':
        commit_message = args.message or 'Some commit'
        github_commit_tool(commit_message)
        print('Committed to Github')
    elif args.command == 'create_file':
        file_contents = args.contents
        file_path = args.path
        create_file_tool(file_contents, file_path)
        print('File created')
    elif args.command == 'read_file':
        file_path = args.path
        contents = read_file_tool(file_path)
        print(contents)

if __name__ == '__main__':
    main()